In [1]:
import os
cwd = os.getcwd()
data_dir = cwd + '/dataset'
os.listdir(data_dir)

['prev_check2_clean.csv',
 'application_test.csv',
 '.DS_Store',
 'HomeCredit_columns_description.csv',
 'POS_CASH_balance.csv',
 'train_set.csv',
 'test_set.csv',
 'credit_card_balance.csv',
 'installments_payments.csv',
 'application_train.csv',
 'bureau.csv',
 'bu_clean.csv',
 'prev_check1_clean.csv',
 'previous_application.csv',
 'HomeCredit_columns_description.xls',
 'bureau_balance.csv',
 'sample_submission.csv']

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns

import lightgbm as lgb
from sklearn.cross_validation import train_test_split,KFold
from sklearn.metrics import auc
from sklearn.model_selection import GridSearchCV

%matplotlib inline
pd.set_option('max_columns',500)
pd.set_option('max_rows',500)

/anaconda3/envs/tensorflow35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [3]:
train_set = pd.read_csv(data_dir + "/train_set.csv",index_col='SK_ID_CURR')
test_set = pd.read_csv(data_dir + "/test_set.csv",index_col='SK_ID_CURR')

In [4]:
y_train = train_set["TARGET"].values
x_train = train_set.drop(["TARGET"],axis = 1).values
x_test = test_set.values

In [5]:
#X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=10)
#train_set.info()
#type(y_train[0])
kf = KFold(len(x_train),n_folds=5)

In [6]:
gbm = lgb.LGBMClassifier(objective='binary',
                         max_depth=8,
                         nthread=4,
                         subsample=0.8715623, #bagging
                         colsample_bytree=0.9497036, #feature fraction
                         reg_alpha=0.041545473, #l1 regularization
                         reg_lambda=0.0735294,  #l2 regularization
                         min_split_gain=0.0222415,
                         min_child_weight=39.3259775, #min data in leaf
                         num_leaves=34,
                         learning_rate=0.03,
                         n_estimators=10000,
                         is_ubalance=True,
                         verbose=-1)

In [25]:
result = np.zeros((len(x_test),5))
oof_result = np.zeros((len(x_train)))
for i,(train_index,valid_index) in enumerate(kf):
    X_train = x_train[train_index] 
    X_valid = x_train[valid_index]
    Y_train = y_train[train_index]
    Y_valid = y_train[valid_index]
    print('Start training {} fold...'.format(i))
    gbm.fit(X_train,Y_train,
            eval_set=[(X_train,Y_train),(X_valid,Y_valid)],
            eval_metric='auc',
            early_stopping_rounds=300,
            verbose=100,
            )
    result[:,i] = gbm.predict(x_test,num_iteration=gbm.best_iteration_)
    oof_result[valid_index] = gbm.predict(X_valid,num_iteration=gbm.best_iteration_)
    

Start training 0 fold...
Training until validation scores don't improve for 300 rounds.


KeyboardInterrupt: 

In [8]:
'''param_grid = {
    'learning_rate':[0.01,0.05,0.1],
    'num_leaves':[20, 30, 40, 50, 60],   
}
gbm_search = GridSearchCV(gbm, param_grid)
gbm_search.fit(X_train,Y_train,
               eval_set=[(X_valid,Y_valid)],
               eval_metric='auc',
               early_stopping_rounds=300,
               verbose=100)
print(gbm_search.best_params_)
gbm.fit(X_train,Y_train,
        eval_set=[(X_train,Y_train),(X_valid,Y_valid)],
        eval_metric='auc',
        verbose=100,
        )'''

"param_grid = {\n    'learning_rate':[0.01,0.05,0.1],\n    'num_leaves':[20, 30, 40, 50, 60],   \n}\ngbm_search = GridSearchCV(gbm, param_grid)\ngbm_search.fit(X_train,Y_train,\n               eval_set=[(X_valid,Y_valid)],\n               eval_metric='auc',\n               early_stopping_rounds=300,\n               verbose=100)\nprint(gbm_search.best_params_)\ngbm.fit(X_train,Y_train,\n        eval_set=[(X_train,Y_train),(X_valid,Y_valid)],\n        eval_metric='auc',\n        verbose=100,\n        )"

In [12]:
trace = go.Scatter(
    y = gbm.feature_importances_,
    x = train_set.columns.values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
        color = gbm.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = train_set.columns.values,
    name = 'LGBM'
)
layout = go.Layout(
        autosize=True,
        title =  'Feature Importances',
        hovermode = 'closest',
        yaxis = dict(
            title = 'Feature Importance',
            ticklen = 5,
            gridwidth = 2
        ),
        showlegend = False
    )
fig = go.Figure(data = [trace],layout = layout)
py.iplot(fig,filename='scatter')

In [13]:
y_pred = np.sum(result,axis = 1)/5

In [14]:
y_pred[y_pred<0] = 0

In [19]:
score = auc(oof_result,y_train,reorder=True)
print(score)


0.5


In [ ]:
result = pd.DataFrame({"SK_ID_CURR":np.array(test_set.index),
                       "TARGET":y_pred
                      })

In [ ]:
result.to_csv('submission.csv',index=False)

In [ ]:
train_test_split?

In [17]:
auc?

In [ ]:
y_pred

In [ ]:
(y_pred<0).any()

In [ ]:
cnt = 123
print('the number is {}'.format(cnt))

In [21]:
print(sum(oof_result))

1677.0


AttributeError: 'DataFrame' object has no attribute 'STATUS'